#Esercizio 1
Using all the Python modules you need compute the following distances:

a. Compute the $L_p$ distance between $(1, 2)$ and $(3, 4)$ for $p = 1, 2, ∞$.

b. Given two objects represented by the tuples $(22, 1, 42, 10)$ and $(20, 0, 36, 8)$:

1. Compute the Euclidean distance between the two objects.
2. Compute the Manhattan distance between the two objects.
3. Compute the Minkowski distance between the two objects, using $h = 3$.
4. Compute the supremum distance ($L_∞$) between the two objects.


In [ ]:
from scipy.spatial.distance import minkowski
from scipy.spatial.distance import chebyshev
import math

# a
obj1 = [1,2]
obj2 = [3,4]

def distance(o1, o2, p):
  # assume len(o1) = len(o2)
  sum = 0
  for x in range(len(o1)):
    sum += math.pow(abs(o1[x] - o2[x]), p)
  result = math.pow(sum, (1/p))
  return result

print(distance(obj1, obj2, 1))
print(distance(obj1, obj2, 2))
print(distance(obj1, obj2, math.inf)) # ?
print(minkowski(obj1, obj2, math.inf))
print('\n')

# b
obj1 = [22,1,42,10]
obj2 = [20,0,36,8]

print(f'Euclidean: {distance(obj1, obj2, 2)}')
print(f'City-block: {distance(obj1, obj2, 1)}')
print(f'L-3: {minkowski(obj1, obj2, 3)}')
print(f'L-inf: {minkowski(obj1, obj2, math.inf)}')

# risultati corretti


4.0
2.8284271247461903
1.0
2.0


Euclidean: 6.708203932499369
City-block: 11.0
L-3: 6.153449493663682
L-inf: 6.0


#Esercizio 2
Download the Ionosphere data set from the UCI Machine Learning Repository, remove the class feature and compute the $L_p$ distance between all pairs of data points, for $p=1,2,∞$.

Compute the contrast measure on the data set for the different norms.

Repeat the exercise after sampling the first $r$ dimensions, where $r$ varies from 1 to the full dimensionality of the data.

In [39]:
import numpy as np
a = [ [23, 26, 20], [22, 28, 29], [19, 30, 24] ]
print(np.max(a))
print(np.min(a))
print(np.mean(a))
print(np.average(a))

30
19
24.555555555555557
24.555555555555557


In [41]:
# soluzione fatta a casa 18-01-2024

import pandas as pd
import numpy as np
from scipy.spatial.distance import minkowski
import math

def contrast(df):
  matrix = df.to_numpy()
  a = matrix.flatten()
  return ((np.max(a) - np.min(a)) / np.mean(a))

def calculateDistances(df, p):
  dist_df = np.zeros((int(df.shape[0]), int(df.shape[0])))
  dist_df = pd.DataFrame(dist_df)
  (rows, columns) = df.shape
  for r1 in range(rows):
    for r2 in range(rows):
      dist_df.iloc[r1].iloc[r2] = minkowski(df.iloc[r1], df.iloc[r2], p)
  return dist_df

labels=list(range(0, 34)) + ['class']
df = pd.read_csv('http://www.lacascia.it/bd2/_ionosphere/ionosphere.data', names=labels)
print(df.shape)
df = df.iloc[:, 0:34] # remove 'class' column

l1_df = calculateDistances(df, 1)
l2_df = calculateDistances(df, 2)
LInf_df = calculateDistances(df, math.inf)

contrast1 = contrast(l1_df)
contrast2 = contrast(l2_df)
contrastInf = contrast(LInf_df)

print(f'contrast L-1: {contrast1}')
print(f'contrast L-2: {contrast2}')
print(f'contrast L-2: {contrastInf}')

(351, 35)
contrast L-1: 2.637098523831358
contrast L-2: 2.448631091359712
contrast L-2: 1.4544022218157644


In [51]:
# with sampling

import random

# create random unique indexes
(rows, colums) = df.shape

indexes = random.sample(range(0, rows), round(rows * 0.2)) # 20 % degli elementi totali
sample_df = df.iloc[indexes]
sample_df.shape

l1_sdf = calculateDistances(sample_df, 1)
l2_sdf = calculateDistances(sample_df, 2)
LInf_sdf = calculateDistances(sample_df, math.inf)

contrast1 = contrast(l1_sdf)
contrast2 = contrast(l2_sdf)
contrastInf = contrast(LInf_sdf)

print(f'sample DF contrast L-1: {contrast1}')
print(f'sample DF contrast L-2: {contrast2}')
print(f'sample DF contrast L-2: {contrastInf}')

sample DF contrast L-1: 2.3715759966514054
sample DF contrast L-2: 2.2308742546302014
sample DF contrast L-2: 1.4946811724722031


In [ ]:
# soluzione fatta in aula

import pandas as pd
import numpy as np
from scipy.spatial.distance import minkowski

# IL CONTRASTO SI CALCOLA RISPETTO ALL'ORIGINE ?
# si e no. Lo puoi fare sia rispetto all origine che "tutti-contro-tutti".
# Per data set piccoli, meglio "tutti-contro-tutti"

def contrast(matrix):
  a = matrix.flatten()
  return ((np.max(a) - np.min(a)) / np.mean(a))

def calculateDistances(data, p):
  distances = np.zeros((351, 351))
  for idx, x in enumerate(data):
    for idy, y in enumerate(data):
      distances[idx, idy] = minkowski(x,y,p)
  return distances

labels=list(range(1, 35)) + ['class']
df = pd.read_csv('http://www.lacascia.it/bd2/_ionosphere/ionosphere.data', names=labels)
data = df.iloc[:, :-1].to_numpy(copy=True)
# print(data.shape)
norm1 = calculateDistances(data, 1)
norm2 = calculateDistances(data, 2)
normInf = calculateDistances(data, math.inf)
contrast1 = contrast(norm1)
contrast2 = contrast(norm2)
contrastInf = contrast(normInf)

print(contrast1)
print(contrast2)
print(contrastInf)


2.637098523831358
2.448631091359712
1.4544022218157644


#Esercizio 3
Caricate il musk dataset, eliminate le prime colonne non numerichee e riducete la dimensionalità con PCA tenendo il 99% della varianza.

1. Effettuate la classificazione con k-NN utilizzando la distanza di Mahalanobis e confrontate l'accuracy con quella ottenuta con la distanza di default (Euclidea).

2. Provate a implementare la match-based similarity ed effettuare di nuovo la classificazione k-NN.

(Suggerimento: vedete dalla documentazione come si può utilizzare il KNeighborsClassifier di sklearn con metriche diverse dalla Euclidea o implementate voi l'algoritmo k-NN)

In [56]:
import pandas as pd
from sklearn.decomposition import PCA

labels=['molecule', 'conformation']+list(range(166)) +['class']
df = pd.read_csv('http://www.lacascia.it/bd2/clean2.data', names=labels)

data = df.iloc[:, 2:-1].to_numpy(copy=True)
target = df.iloc[:, -1].to_numpy(copy=True)
print(target.shape, data.shape)

pca = PCA(n_components=0.99)
data_n = pca.fit(data).transform(data)
print(data_n.shape)

(6598,) (6598, 166)
(6598, 70)


In [57]:
import numpy as np
from numpy.linalg import inv
# calcolo l'inversa della matrice di covarianza dei dati necessaria per la distanza di Mahalanobis
C=np.cov(data_n.T)
CI = inv(C)
CI.shape

(70, 70)

In [59]:
# Faccio lo split fra train e test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_n, target, random_state=11)

print(X_train.shape)
print(X_test.shape)

(4948, 70)
(1650, 70)


In [60]:
# Calcolo la matrice delle distanze per il train
from scipy.spatial.distance import mahalanobis
dist_train = np.zeros((X_train.shape[0], X_train.shape[0]))
for i in range(X_train.shape[0]):
  for j in range(X_train.shape[0]):
#    dist_train[i, j] = minkowski(X_train[i, :], X_train[j, :], 2)
    dist_train[i, j] = mahalanobis(X_train[i, :], X_train[j, :], CI)
print(f'dist_train: {dist_train.shape}')

# Calcolo la matrice delle distanze del test dal train
dist_test = np.zeros((X_test.shape[0], X_train.shape[0]))
for i in range(X_test.shape[0]):
  for j in range(X_train.shape[0]):
#    dist_test[i, j] = minkowski(X_test[i, :], X_train[j, :], 2)
    dist_test[i, j] = mahalanobis(X_test[i, :], X_train[j, :], CI)
print(f'dist_test: {dist_test.shape}')

# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

dist_train: (4948, 4948)
dist_test: (1650, 4948)
Accuracy: 96.79%


# NOTA: vedi solEs05 per la versione con match-based similarity

#Esercizio 4
Implementate la shared nearest-neighbor similarity (scegliete voi il numero di vicini da considerare)
ed effettuate nuovamente la classificazione k-NN.
Se l'accuratezza del classificatore migliora cosa potete ipotizzare sulla distribuzione dei vostri dati.

In [61]:
# Uso sklearn.neighbors per trovare i k vicini di due punti, vedo quanti
# sono in comune e ritorno la distanza definita come 1-similarity_normalizzata)
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X_train)

def shared_nearest(x1, x2):
  res=neigh.kneighbors([x1, x2], return_distance=False)
  count=0
  for v in res[0]:
    if v in res[1]:
      count += 1
  return 1-count/10

print(shared_nearest(X_test[50], X_test[10]))
print(shared_nearest(X_test[50], X_test[50]))

1.0
0.0


In [65]:
# Calcolo la matrice delle distanze per il train
# Prendo solo parte del dataset per motivi di tempo di calcolo
X_train = X_train[:500]
y_train = y_train[:500]
X_test = X_test[:150]
y_test = y_test[:150]

In [66]:
# Calcolo la matrice delle distanze per il train
import numpy as np

size = int(X_train.shape[0])
dist_train = np.zeros((size, size))
for i in range(size):
  for j in range(size):
    dist_train[i, j] = shared_nearest(X_train[i, :], X_train[j, :])
print(f'dist_train: {dist_train.shape}')


dist_train: (500, 500)


In [69]:
# Calcolo la matrice delle distanze del test dal train
size_train = int(X_train.shape[0])
size_test = int(X_test.shape[0])
dist_test = np.zeros((size_test, size_train))
for i in range(size_test):
  for j in range(size_train):
    dist_test[i, j] = shared_nearest(X_test[i, :], X_train[j, :])
print(f'dist_test: {dist_test.shape}')


dist_test: (100, 500)


In [71]:
# Costruisco il modello e ne valuto le performance
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, metric='precomputed')
knn.fit(X=dist_train, y=y_train)

print(f'Accuracy: {knn.score(dist_test, y_test):.2%}')

Accuracy: 95.00%
